# 스타일 전이(Style Transfer)

* 그림은 내용과 스타일의 복잡한 상호작용을 표현한다. 한편 사진은 원근과 빛의 조합이다. 이 둘을 합쳤을 때의 결과는 매우 놀랍다. 이 과정을 예술적 스타일 전이라고 한다. 

* 논문 “A Neural Algorithm for Artistic Style(예술적 스타일의 신경 알고리즘)”(https://arxiv.org/abs/1508.06576)에서 제시한 전이학습 알고리즘의 구현에 대해서 논의할 것이다.

## 신경스타일전이
* 신경 스타일 전이는 특정 타깃 이미지에 참조 이미지의 스타일을 적용하면서 타깃 이미지의 원래 내용은 변경되지 않도록 하는 과정이다. 
* 여기서 주된 목적은 원본 타깃 이미지의 내용을 유지하면서 타깃 이미지에 참조 이미지의 스타일을 겹치거나 적용하는 것이다.
* 이 개념을 수학적으로 표현하려면 다음과 같은 3가지 이미지를 고려해야 한다. 원본 내용(c로 표시), 참조 스타일(s로 표시), 생성된 이미지(g로 표시)가 그것이다. c와 g가 내용면에서 얼마나 다른 이미지인지 측정할 방법이 필요하다. 또한 출력 스타일의 특성이라는 측면에서 출력 이미지는 스타일 이미지와 차이가 크지 않아야 한다.

In [1]:
%matplotlib inline

## 이미지 전처리

In [2]:
import numpy as np
from keras.applications import vgg16
from keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path, height=None, width=None):
    height = 400 if not height else height
    width = width if width else int(width * height / height)
    img = load_img(image_path, target_size=(height, width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

def deprocess_image(x):
    #  평균 픽셀로 zero-center 제거
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

Using TensorFlow backend.
/home/dyanos/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dyanos/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dyanos/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dyanos/anaconda3/lib/python3.7/site-packages/tenso

In [3]:
from keras import backend as K

# 변환하려는 이미지의 경로
TARGET_IMG = 'lotr.jpg'
# 스타일 이미지의 경로
REFERENCE_STYLE_IMG = 'pattern1.jpg'

width, height = load_img(TARGET_IMG).size
img_height = 480
img_width = int(width * img_height / height)


target_image = K.constant(preprocess_image(TARGET_IMG, height=img_height, width=img_width))
style_image = K.constant(preprocess_image(REFERENCE_STYLE_IMG, height=img_height, width=img_width))

# 생성될 이미지의 플레이스홀더
generated_image = K.placeholder((1, img_height, img_width, 3))

# 3개의 이미지를 단일 배치로 결합
input_tensor = K.concatenate([target_image,
                              style_image,
                              generated_image], axis=0)

In [4]:
print(generated_image)

Tensor("Placeholder:0", shape=(1, 480, 2012, 3), dtype=float32)


In [5]:
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)

In [6]:
layers = {l.name: l.output for l in model.layers}
layers

{'input_1': <tf.Tensor 'concat:0' shape=(3, 480, 2012, 3) dtype=float32>,
 'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 480, 2012, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 480, 2012, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 240, 1006, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 240, 1006, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 240, 1006, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 120, 503, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 120, 503, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 120, 503, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 120, 503, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(3, 60, 251, 256) dtype=float32>,
 'block4_con

# 손실함수 구축

## 내용 손실 


CNN 기반 모델에서 최상위 계층의 활성화는 더 전역적이고 추상적인 정보(예: 얼굴과 같은 고수준 구조)를 포함하며, 하단 층은 이미지에 대한 국소 정보(예: 눈, 코, 에지, 코너와 같은 저수준 구조)를 포함한다. 여기서는 이미지 내용에 대한 올바른 표현을 포착하기 위해 CNN의 상층을 활용한다. 따라서 사전 훈련된 VGG-16 모델을 사용할 때의 내용 손실은 타깃 이미지를 통해 계산된 상위 층(주어진 특성 표현)의 활성화 계산과 생성된 이미지를 통해 계산된 동일한 층의 활성화 사이의 L2 노름(스케일과제곱의 유클리드 거리)으로 정의할 수 있다. 보통 CNN의 상위 층으로부터 이미지의 내용과 관련된 특성적 표현을 받는다고 가정하면 생성된 이미지는 기본 타깃 이미지와 유사해 보일 것이다. 다음 코드는 내용 손실을 계산하는 함수다.

In [7]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

## 스타일 손실

논문 “A Neural Algorithm of Artistic Style” (https://arxiv.org/abs/1508.06576)에 따라 여기서는 그램 행렬을 사용할 것이다.

In [8]:
def style_loss(style, combination, height, width):
    
    def build_gram_matrix(x):
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
        gram_matrix = K.dot(features, K.transpose(features))
        return gram_matrix

    S = build_gram_matrix(style)
    C = build_gram_matrix(combination)
    channels = 3
    size = height * width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

## 총 변동 손실

스타일과 내용 손실을 줄이기 위한 최적화는 때때로 너무 픽셀화되고 노이즈가 있는 출력으로 이어지기도 한다. 이를 극복하기 위해 총 변동 손실이 도입됐다. 총 변동 손실(total variation loss)은 ‘정규화 손실’과 비슷하다. 발생할 이미지를 공간 연속적이고 매끄럽게 보이기 위해 도입됐기 때문에 화소에 노이즈가 과다하게 나타나는 것을 방지한다.

In [9]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

## 총 손실 함수

신경 전이를 위한 총 손실 함수의 구성 요소를 정의했으므로 다음 단계는 구성 요소를 결합하는 것이다. 내용 및 스타일 정보는 CNN 네트워크의 다른 깊이에서 포착되므로 적절한 층에 각각의 손실 유형을 적용하고 계산해야 한다. 스타일 손실을 위해서 합성곱층 1부터 5까지를 취하고 각 층에 적절한 가중치를 설정한다.

In [10]:
# 가중 평균 손실 함수에 대한 가중치
content_weight = 0.05
total_variation_weight = 1e-4


content_layer = 'block4_conv2'
style_layers = ['block{}_conv2'.format(o) for o in range(1,6)]
style_weights = [0.1, 0.15, 0.2, 0.25, 0.3]

# 전체 손실 값 초기화
loss = K.variable(0.)

# 내용 손실 추가
layer_features = layers[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + (content_weight * content_loss(target_image_features,
                                      combination_features))

# 스타일 손실 추가
for layer_name, sw in zip(style_layers, style_weights):
    layer_features = layers[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features, 
                    height=img_height, width=img_width)
    loss = loss + sl*sw

# 전체 편차의 손실 값 추가
loss = loss + (total_variation_weight * total_variation_loss(generated_image))

In [11]:
# 손실로 생성된 이미지의 기울기를 구한다.
grads = K.gradients(loss, generated_image)[0]

# 현재 손실과 현재 기울기를 구하는 함수
fetch_loss_and_grads = K.function([generated_image], [loss, grads])


class Evaluator(object):

    def __init__(self, height=None, width=None):
        self.loss_value = None
        self.grads_values = None
        self.height = height
        self.width = width

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, self.height, self.width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator(height=img_height, width=img_width)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
from scipy.optimize import fmin_l_bfgs_b
#from scipy.misc import imsave # scipy 1.3.0에서는 deprecated된 것이라 아래 것으로 대체
from imageio import imwrite
import time

result_prefix = 'st_res_'+TARGET_IMG.split('.')[0]
iterations = 20

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(TARGET_IMG, height=img_height, width=img_width)
x = x.flatten()

for i in range(iterations):
    print('Start of iteration', (i+1))
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    if (i+1) % 5 == 0 or i == 0:
        # Save current generated image only every 5 iterations
        img = x.copy().reshape((img_height, img_width, 3))
        img = deprocess_image(img)
        fname = result_prefix + '_iter%d.png' %(i+1)
        imwrite(fname, img)
        print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i+1, end_time - start_time))

Start of iteration 1
Current loss value: 51848086000.0
Image saved as st_res_lotr_iter1.png
Iteration 1 completed in 20s
Start of iteration 2
Current loss value: 50630926000.0
Iteration 2 completed in 14s
Start of iteration 3
Current loss value: 49781457000.0
Iteration 3 completed in 14s
Start of iteration 4


In [ ]:
from skimage import io
from glob import glob
from matplotlib import pyplot as plt

In [ ]:
content_image = io.imread('lotr.jpg')
style_image = io.imread('pattern1.jpg')

iter1 = io.imread('st_res_lotr_iter1.png')
iter5 = io.imread('st_res_lotr_iter5.png')
iter10 = io.imread('st_res_lotr_iter10.png')
iter15 = io.imread('st_res_lotr_iter15.png')
iter20 = io.imread('st_res_lotr_iter20.png')

In [ ]:
fig = plt.figure(figsize = (15, 6))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(content_image)
t1 = ax1.set_title('Content Image')
ax2 = fig.add_subplot(1,2, 2)
ax2.imshow(style_image)
t2 = ax2.set_title('Style Image')

In [ ]:
fig = plt.figure(figsize = (15, 15))

ax1 = fig.add_subplot(6,3, 1)
ax1.imshow(content_image)
t1 = ax1.set_title('Original')

ax1 = fig.add_subplot(6,3, 2)
ax1.imshow(iter1)
t1 = ax1.set_title('Iteration 1')

ax1 = fig.add_subplot(6,3, 3)
ax1.imshow(iter5)
t1 = ax1.set_title('Iteration 5')

ax1 = fig.add_subplot(6,3, 4)
ax1.imshow(iter10)
t1 = ax1.set_title('Iteration 10')

ax1 = fig.add_subplot(6,3, 5)
ax1.imshow(iter15)
t1 = ax1.set_title('Iteration 15')

ax1 = fig.add_subplot(6,3, 6)
ax1.imshow(iter20)
t1 = ax1.set_title('Iteration 20')

plt.tight_layout()
fig.subplots_adjust(top=0.95)
t = fig.suptitle('LOTR Scene after Style Transfer')

In [ ]:
fig = plt.figure(figsize = (15, 15))
ax1 = fig.add_subplot(6,3, 1)
ax1.imshow(content_image)
t1 = ax1.set_title('Original')

gen_images = [iter1,iter5, iter10, iter15, iter20]

for i, img in enumerate(gen_images):
    ax1 = fig.add_subplot(6,3,i+1)
    ax1.imshow(content_image)
    t1 = ax1.set_title('Iteration {}'.format(i+5))

plt.tight_layout()
fig.subplots_adjust(top=0.95)
t = fig.suptitle('LOTR Scene after Style Transfer')

In [ ]:
fig = plt.figure(figsize = (20, 20))

ax1 = fig.add_subplot(2,1, 1)
ax1.imshow(content_image)
t1 = ax1.set_title('Original Image')


ax1 = fig.add_subplot(2,1, 2)
ax1.imshow(iter20)
t1 = ax1.set_title('Stylized Image')